# Практическая работа 1
## Выполнил Гневнов А.Е., ИВТ 2.1
## Подготовка данных и обучение модели

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
import pickle
import warnings

warnings.filterwarnings('ignore')
sns.set_style('whitegrid')

### 1. Загрузка и анализ данных

In [ ]:
df = pd.read_csv('WineQT.csv')
print("Размерность данных:", df.shape)

In [ ]:
# Первичный анализ
print("\nПервые 5 строк:")
display(df.head())

print("\nИнформация о данных:")
display(df.info())

print("\nСтатистика:")
display(df.describe())

In [ ]:
# Анализ целевой переменной
plt.figure(figsize=(10, 6))
sns.countplot(x='quality', data=df)
plt.title('Распределение качества вина')
plt.xlabel('Качество')
plt.ylabel('Количество')
plt.show()

In [ ]:
# Корреляционный анализ
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Корреляционная матрица')
plt.show()

### 2. Подготовка данных

In [ ]:
# Разделение на признаки и целевую переменную
X = df.drop(['quality', 'Id'], axis=1)
y = df['quality']

In [ ]:
# Разделение на обучающую, валидационную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.176, random_state=42, stratify=y_train
)

print(f"Обучающая выборка: {X_train.shape[0]} записей")
print(f"Валидационная выборка: {X_val.shape[0]} записей")
print(f"Тестовая выборка: {X_test.shape[0]} записей")

In [ ]:
# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

# Сохранение скейлера
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

### 3. Обучение моделей

In [ ]:
# Инициализация моделей
models = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'DecisionTree': DecisionTreeClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'MLP': MLPClassifier(random_state=42, max_iter=1000)
}

In [ ]:
# Обучение и оценка моделей
results = {}

for name, model in models.items():
    print(f"\nОбучение модели: {name}")
    model.fit(X_train_scaled, y_train)
    
    # Предсказания на валидационной выборке
    y_pred = model.predict(X_val_scaled)
    
    # Расчет метрик
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred, average='weighted')
    
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'f1_score': f1
    }
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(classification_report(y_val, y_pred))

In [ ]:
# Сводная таблица результатов
results_df = pd.DataFrame.from_dict(results, orient='index')
results_df.drop(columns='model', inplace=True)
results_df.sort_values(by='f1_score', ascending=False, inplace=True)

display(results_df)

In [ ]:
# Визуализация результатов
plt.figure(figsize=(10, 6))
results_df.plot(kind='bar', y=['accuracy', 'f1_score'], figsize=(12, 6))
plt.title('Сравнение производительности моделей')
plt.ylabel('Оценка')
plt.xticks(rotation=15)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

### 5. Тюнинг лучшей модели

In [ ]:
# Выбор лучшей модели
best_model_name = results_df.index[0]
best_model = results[best_model_name]['model']
print(f"Лучшая модель: {best_model_name}")

In [ ]:
# Параметры для GridSearchCV
param_grids = {
    'RandomForest': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'XGBoost': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5]
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf']
    },
    'KNN': {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance']
    },
    'MLP': {
        'hidden_layer_sizes': [(50,), (100,)],
        'alpha': [0.0001, 0.001]
    }
}

In [ ]:
# Тюнинг выбранной модели
if best_model_name in param_grids:
    print(f"\nТюнинг модели: {best_model_name}")
    grid_search = GridSearchCV(
        estimator=best_model,
        param_grid=param_grids[best_model_name],
        scoring='f1_weighted',
        cv=3,
        n_jobs=-1,
        verbose=1
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    # Обновление лучшей модели
    best_model = grid_search.best_estimator_
    print(f"Лучшие параметры: {grid_search.best_params_}")
    
    # Оценка после тюнинга
    y_val_pred = best_model.predict(X_val_scaled)
    tuned_f1 = f1_score(y_val, y_val_pred, average='weighted')
    print(f"F1 после тюнинга: {tuned_f1:.4f}")

### 6. Оценка на тестовых данных

In [ ]:
# Финальная оценка
y_test_pred = best_model.predict(X_test_scaled)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred, average='weighted')

print(f"\nРезультаты на тестовых данных:")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(classification_report(y_test, y_test_pred))

In [ ]:
# Матрица ошибок
plt.figure(figsize=(10, 8))
cm = confusion_matrix(y_test, y_test_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Матрица ошибок')
plt.xlabel('Предсказанные значения')
plt.ylabel('Истинные значения')
plt.show()

### 7. Сохрананяем модель

In [ ]:
# Сохранение лучшей модели
with open('best_wine_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)
print("Модель сохранена как 'best_wine_model.pkl'")

### 8. Пример использования модели и скейлера

In [ ]:
def predict_wine_quality(features):
    """Предсказывает качество вина на основе входных признаков"""
    # Загрузка модели и скейлера
    with open('best_wine_model.pkl', 'rb') as f:
        model = pickle.load(f)
    with open('scaler.pkl', 'rb') as f:
        scaler = pickle.load(f)
    
    # Преобразование признаков
    scaled_features = scaler.transform([features])
    
    # Предсказание
    prediction = model.predict(scaled_features)
    return prediction[0]

# Пример использования
sample_wine = {
    'fixed acidity': 7.4,
    'volatile acidity': 0.70,
    'citric acid': 0.00,
    'residual sugar': 1.9,
    'chlorides': 0.076,
    'free sulfur dioxide': 11.0,
    'total sulfur dioxide': 34.0,
    'density': 0.9978,
    'pH': 3.51,
    'sulphates': 0.56,
    'alcohol': 9.4
}

# Убедимся, что порядок признаков соответствует обучению
features_ordered = [sample_wine[col] for col in X.columns]
predicted_quality = predict_wine_quality(features_ordered)
print(f"\nПредсказанное качество вина: {predicted_quality}")